## It is often the case that we only want to use part of a input sequence. In the case of a sentence, we would like to "cover" later parts of the sentence for next token prediction, otherwise it will be trivial for the model to grab the prediction token from the input sentence. TODO: expand this on training algorithm that necessitates this, and why use -inf to achieve this, cool comparison trick to make masks; For now straight to the point -- train masked model

In [ ]:
# what are we having from the data??
